In [1]:
import pandas as pd
import datetime as dt
import footballData as fd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import time
import Elo as ELO
import seaborn as sns

In [2]:
df1888=pd.read_csv('englandthesis.csv')
startdate=1958
fulldf=df1888[df1888['Season']>=startdate]
fulldf=fulldf.sort_values('Date')
fulldf=fulldf.reset_index()
fulldf=fulldf.replace('Milton Keynes Dons', 'Wimbledon')

def CalendarYear(Date):
    return int(Date.split('-')[0])
CalendarYear(fulldf['Date'][0])
def Month(Date):
    return int(Date.split('-')[1])
fulldf['CalendarYear']=fulldf.apply(lambda row: CalendarYear(row['Date']), axis=1)
fulldf['Month']=fulldf.apply(lambda row: Month(row['Date']), axis=1)
import datetime
from datetime import datetime
def DT(Date):
    return datetime.strptime(Date, '%Y-%m-%d')
fulldf['Date']=fulldf.apply(lambda row: DT(row['Date']), axis=1)

def to_integer(current_time):
    return (current_time-fulldf['Date'][0]).days
fulldf['Day#']=fulldf.apply(lambda row: to_integer(row['Date']), axis=1)
fulldf

import copy
from copy import deepcopy
def g(sigma2, q):
    return 1/np.sqrt(1+3*q**2*sigma2/(np.pi**2))

def E(r, rj, g, Home, HomeAway):
    return 1/(1+10**(-g*(r-rj+HomeAway*Home)/400))

def Res(HomeG, AwayG):
    if(HomeG>AwayG):
        return 1
    elif(AwayG>HomeG):
        return 0
    else:
        return .5
def insideD(g, E):
    return E*(1-E)*(g**2)

def SeasonBoost(rd, b, maxrd):
    return min(maxrd, np.sqrt(rd**2+b))

def d2(q, dlist):
    if dlist!=0:
        return 1/(q**2*dlist)
    else:
        return 10**10

def ratingupdate(rating, q, d, RD, sums):
    return rating+(q/(1/RD**2+1/d))*sums      
def rdupdate(rd, d):
    return np.sqrt(1/(1/rd**2+1/d))

def timeupdate(c, rd, maxrd):
    return min(np.sqrt(rd**2+c**2), maxrd)

def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)

initdictionary={}
initvalues=[1500,1500,1500,1500]
initialrows=[]
Rd=350

for i in range(len(fulldf['home'].unique())):
    Team=fulldf['home'].unique()[i]
    Teamdf=fulldf[fulldf['home']==Team].reset_index()
    Tier=int(Teamdf['tier'][0])
    LastMatch=fulldf['Day#'][0]
    dcalc=0
    scalc=0
    initialrow=[Team,initvalues[Tier-1], Rd,Tier,  dcalc,scalc ]
    initialrows.append(initialrow)
#Needs to be rewritten to optimize using BuildEloDict Function
for a in range(len(initialrows)):
    initdictionary[initialrows[a][0]]=initialrows[a][1:]


/Users/andrewpuopolo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
def Glickologlikboost(params):
 #   yearleagues=[]
  #  rows2=[]
   # rows3=[]
    ratingperiodlength=7.0
    c=params['c']
    h=80
    hk=params['hk']
    b=params['b']
    #minrd=params['minrd']
    maxrd=params['maxrd']
    curtime=time.time()
    rows=len(fulldf)
    q=np.log(10)/400
    logliks=0.0
    hburn=9
    startyear=fulldf['Season'].values[0]
    burn=41
    #print(startyear)
    year=copy.deepcopy(startyear)
    startday=fulldf['Season']
    Glickodictionary=copy.deepcopy(initdictionary)
    for key in Glickodictionary:
        Glickodictionary[key][1]=maxrd
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    games=0
   # unitedrows=[]
    #print(Glickodictionary)
    while day<=totaltime:
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        
        if len(ratingperioddf!=0):
            for key in Glickodictionary:
                Glickodictionary[key][1]=timeupdate(c, Glickodictionary[key][1], maxrd)
            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                if(row['Season']!=year):
                    for key in Glickodictionary:
                        Glickodictionary[key][1]=SeasonBoost(Glickodictionary[key][1], b**2, maxrd)
                   # yearleagues.append([row['Season'], np.mean(firsts), np.mean(seconds), np.mean(thirds), np.mean(fourths)])
                         
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                Homestats=Glickodictionary[Home]
                Awaystats=Glickodictionary[Away]
                HomeRating=Homestats[0]
                AwayRating=Awaystats[0]
                HomeRd=Homestats[1]
                AwayRd=Awaystats[1]
                HomeG=g(AwayRd**2, q)
                AwayG=g(HomeRd**2, q)
                pG=g(HomeRd**2+AwayRd**2, q)
                HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
                AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
                pE=E(HomeRating, AwayRating, pG, h, 1)
                HRes=Res(HomeGoals, AwayGoals)
                ARes=Res(AwayGoals, HomeGoals)

                Glickodictionary[Home][4]+=HomeG*(HRes-HomeE)
                Glickodictionary[Away][4]+=AwayG*(ARes-AwayE)
                Glickodictionary[Home][3]+=insideD(HomeG, HomeE)
                Glickodictionary[Away][3]+=insideD(AwayG, AwayE)
                Rs+=HRes
                Es+=pE
               # row=[Home, Away,date, year,tier, HomeRating, AwayRating, h, HomeRd, AwayRd, pG, HomeGoals, AwayGoals, HRes, pE]
               # rows2.append(row)
                if(year>startyear+burn):
                    logliks+=lik(pE, HRes)
                    games+=1

            if(year>startyear+hburn):
                h+=hk*(Rs-Es)
                
            
            

            for key in Glickodictionary:
                Glickod=d2(q, Glickodictionary[key][3])
                Glickodictionary[key][0]=ratingupdate(Glickodictionary[key][0],q, Glickod, Glickodictionary[key][1], Glickodictionary[key][4])
                Glickodictionary[key][1]=rdupdate(Glickodictionary[key][1], Glickod)
                Glickodictionary[key][3]=0
                Glickodictionary[key][4]=0
            #unitedrows.append([date, copy.deepcopy(Glickodictionary['Manchester United'][0]),copy.deepcopy(Glickodictionary['Manchester United'][1]) ])
        #rows3.append([date, Glickodictionary['Tottenham Hotspur'][1]])  
        day+=ratingperiodlength
    print(params ,logliks,games, Glickodictionary['Tottenham Hotspur'][0],h, time.time()-curtime)
    #print(Glickodictionary)
    return -1*logliks

In [ ]:
from hyperopt import hp
from hyperopt import tpe
    # Algorithm
tpe_algorithm = tpe.suggest

from hyperopt import Trials
    # Trials object to track progress
bayes_trials = Trials()

from hyperopt import fmin

# Algorithm
tpe_algorithm = tpe.suggest

# Trials object to track progress
bayes_trials = Trials()


MAX_EVALS = 3000

# Optimize
#bests=[]

space = {
        #'h': hp.uniform('h', 50, 150),
        #get rid of this for now
        #'minrd': hp.uniform('minrd', 0, 60),
        'hk': hp.uniform('hk', 0, .5),
        'maxrd': hp.uniform('maxrd', 0,350),
    'b': hp.uniform('b',0, 100),
    'c': hp.uniform('c',0,20)
        #'mb': hp.uniform('mb', 0, 5000)
    }



MAX_EVALS = 4000

best2000 = fmin(fn = Glickologlikboost, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)

{'b': 95.90049379636048, 'c': 13.724447971759623, 'hk': 0.18134445230313195, 'maxrd': 40.14758830665357} -22893.093511 34612 1883.48495968 48.4924275998 29.190104007720947
{'b': 22.565543432959434, 'c': 17.399825176054478, 'hk': 0.484930300719502, 'maxrd': 343.10400020841496} -23001.8883587 34612 2138.6955704 60.2128122955 30.041579008102417
{'b': 67.49469951436441, 'c': 11.853788709235129, 'hk': 0.14325197274615997, 'maxrd': 71.12704591444226} -22908.9615874 34612 2087.2477543 51.0137570026 33.530314922332764
{'b': 18.735109533850057, 'c': 10.162711316466531, 'hk': 0.11696456649328574, 'maxrd': 95.05725270136574} -22876.2676293 34612 2050.52187388 50.8407213055 32.219369888305664
{'b': 75.47860013434484, 'c': 0.3150312932074639, 'hk': 0.11507292121166501, 'maxrd': 28.39101116278157} -22990.8768021 34612 1748.86250562 47.5639233839 32.33148169517517
{'b': 96.01612448793102, 'c': 7.219177381353545, 'hk': 0.32971584250980496, 'maxrd': 203.88434832654832} -22980.0165967 34612 2103.9114121

{'b': 16.917571264933976, 'c': 15.24284611692379, 'hk': 0.3155020281684859, 'maxrd': 54.75657556764434} -22871.0260439 34612 1999.98679971 52.674336962 26.452839851379395
{'b': 66.45717886345275, 'c': 5.374740717940642, 'hk': 0.34810083696757543, 'maxrd': 158.2919920175785} -22884.6694376 34612 2047.99466215 54.3745352225 26.62994408607483
{'b': 30.867835029306512, 'c': 2.5239817977945433, 'hk': 0.2304608146649695, 'maxrd': 36.99787056793738} -22916.9318854 34612 1837.2507323 48.8919372751 27.459010124206543
{'b': 12.468752386247743, 'c': 7.863428384502995, 'hk': 0.03948823547012753, 'maxrd': 287.11264593727935} -22885.4351951 34612 1993.03165851 59.058349624 27.768041849136353
{'b': 0.34314523166515265, 'c': 3.6601408727169207, 'hk': 0.4541226795060602, 'maxrd': 179.35774727239772} -22906.9533539 34612 1847.76041922 54.5013836886 27.95905900001526
{'b': 6.844599080800457, 'c': 1.9894684637682145, 'hk': 0.12884631362491816, 'maxrd': 204.55611386044313} -22972.0557456 34612 1771.2551128

In [14]:
best1st

{'b': 20.09392286013549,
 'c': 5.572492971419114,
 'hk': 0.02366983144825603,
 'maxrd': 71.92571394567331}

In [17]:
Glickologlikboost(best1st)


{'b': 20.09392286013549, 'c': 5.572492971419114, 'hk': 0.02366983144825603, 'maxrd': 71.92571394567331} -3983.07974274 6460 1955.00121314 65.3582757349 25.19789719581604


3983.0797427369494

In [15]:
best={'b': 1789.3772474425973,
 'c': 2.9038831973202504,
 'hk': 0.2827192268297882,
 'maxrd': 111.6204164961327}
Glickologlikboost(best)

{'b': 1789.3772474425973, 'c': 2.9038831973202504, 'hk': 0.2827192268297882, 'maxrd': 111.6204164961327} -44179.5843171 67126 1966.94808597 31.61396098136902


44179.584317129978

In [173]:
rows=[]
qs=np.log(10)/400
for i in range(1, 501):
    rows.append([i, g(i**2, qs)])
print(rows)
columns=['sigma', 'g(sigma^2)']
gdf=pd.DataFrame(rows, columns=columns)
plt.plot(gdf['sigma'], gdf['g(sigma^2)'])

[[1, 0.99999496383874353], [2, 0.99997985581149496], [3, 0.9999546772876996], [4, 0.99991943054938481], [5, 0.99987411879058385], [6, 0.99981874611653232], [7, 0.99975331754263563], [8, 0.99967783899320817], [9, 0.99959231729998343], [10, 0.99949676020039735], [11, 0.99939117633564711], [12, 0.99927557524852206], [13, 0.99914996738101203], [14, 0.9990143640716922], [15, 0.9988687775528855], [16, 0.99871322094760639], [17, 0.99854770826628259], [18, 0.99837225440326338], [19, 0.998186875133109], [20, 0.9979915871066678], [21, 0.9977864078469415], [22, 0.99757135574473965], [23, 0.99734645005412759], [24, 0.99711171088766726], [25, 0.99686715921145597], [26, 0.99661281683996417], [27, 0.99634870643067353], [28, 0.99607485147851971], [29, 0.99579127631014352], [30, 0.99549800607794814], [31, 0.99519506675397251], [32, 0.99488248512357647], [33, 0.9945602887789462], [34, 0.99422850611242042], [35, 0.99388716630964002], [36, 0.9935362993425263], [37, 0.99317593596208997], [38, 0.99280610769

In [175]:
zs=[0, 50, 100, 200, 300, 500, 1000]
rows2=[]
q=np.log(10)/400
for i in range(1250):
    ig=g(i**2, q)
    for j in zs:
        iE=E(j, 0, ig, 0, 0)
        rows2.append([i, j, iE])
columns=['Rating Deviation', 'Rating Difference', 'E']
ratingdevE=pd.DataFrame(rows2, columns=columns)

In [176]:
ratingdif0=ratingdevE[ratingdevE['Rating Difference']==0].reset_index()
ratingdif50=ratingdevE[ratingdevE['Rating Difference']==50].reset_index()
ratingdif100=ratingdevE[ratingdevE['Rating Difference']==100].reset_index()
ratingdif200=ratingdevE[ratingdevE['Rating Difference']==200].reset_index()
ratingdif300=ratingdevE[ratingdevE['Rating Difference']==300].reset_index()
ratingdif500=ratingdevE[ratingdevE['Rating Difference']==500].reset_index()
ratingdif1000=ratingdevE[ratingdevE['Rating Difference']==1000].reset_index()

In [191]:
plt.plot(ratingdif1000['Rating Deviation'], ratingdif1000['E'])
plt.plot(ratingdif500['Rating Deviation'], ratingdif500['E'])
plt.plot(ratingdif300['Rating Deviation'], ratingdif300['E'])
plt.plot(ratingdif200['Rating Deviation'], ratingdif200['E'])
plt.plot(ratingdif100['Rating Deviation'], ratingdif100['E'])
plt.plot(ratingdif50['Rating Deviation'], ratingdif50['E'])
plt.plot(ratingdif0['Rating Deviation'], ratingdif0['E'])





plt.xlabel('Rating Deviation',fontsize=14)
plt.ylabel('Expectation',fontsize=14)
plt.title('Expectation By Difference In Rating and Rating Deviation',fontsize=16)
plt.legend(('1000 Rating Points', '500 Rating Points', '300 Rating Points', '200 Rating Points', '100 Rating Points', '50 Rating Points', '0 Rating Points'),
         fontsize=8.8 )
plt.savefig('DeviationPointsE.png')
#plt.show()

In [30]:
def E(r, rj, g, Home, HomeAway):
    return 1/(1+10**(-g*(r-rj+HomeAway*Home)/400))

In [196]:
Unitedrows=Glickologlikboost(bestnew)

{'b': 41.78112448478919, 'c': 3.507640285224943, 'hk': 0.28147894353485076, 'maxrd': 109.05389727014936} -44179.0204871 67126 1969.04766193 24.923181772232056


In [197]:
Unitedrows

[[Timestamp('1958-08-28 00:00:00'), 1555.3946798996024, 100.91138918641684],
 [Timestamp('1958-09-04 00:00:00'), 1524.1870558564074, 94.903775497735964],
 [Timestamp('1958-09-12 00:00:00'), 1529.6524054817958, 89.401922455905535],
 [Timestamp('1958-09-18 00:00:00'), 1550.288890358386, 84.522018845416483],
 [Timestamp('1958-09-25 00:00:00'), 1541.2438461163888, 82.838736307054504],
 [Timestamp('1958-10-02 00:00:00'), 1539.6874201638129, 80.804639928485543],
 [Timestamp('1958-10-09 00:00:00'), 1506.0838875549657, 77.162547320446407],
 [Timestamp('1958-10-17 00:00:00'), 1503.9415675987284, 75.536462455432329],
 [Timestamp('1958-10-22 00:00:00'), 1488.9691821755375, 73.985297848347301],
 [Timestamp('1958-10-31 00:00:00'), 1472.6268250447231, 72.544485609090998],
 [Timestamp('1958-11-01 00:00:00'), 1489.6186726481851, 71.224970643773759],
 [Timestamp('1958-11-08 00:00:00'), 1472.4555374277759, 70.009694587967942],
 [Timestamp('1958-11-15 00:00:00'), 1464.3938162823329, 68.981486837905706],


In [198]:
columns=['TimeStamp', 'Rating', 'Rating Deviation']
Uniteddf=pd.DataFrame(Unitedrows, columns=columns)

In [199]:
Uniteddf['Year']=Uniteddf['TimeStamp'].dt.year

In [200]:
United2014=Uniteddf[Uniteddf['Year']==2014]

In [201]:
United2014.to_csv('United2014b.csv')

In [25]:
absolutevals=Glickologlikboost(best)

{'b': 1789.3772474425973, 'c': 2.9038831973202504, 'hk': 0.2827192268297882, 'maxrd': 111.6204164961327} -44179.5843171 67126 1966.94808597 25.486151933670044


In [26]:
np.mean(absolutevals)

115.49538940930823

In [195]:
def Glickologlikboost(params):
 #   yearleagues=[]
  #  rows2=[]
   # rows3=[]
    ratingperiodlength=7.0
    c=params['c']
    h=80
    hk=params['hk']
    b=params['b']
    #minrd=params['minrd']
    maxrd=params['maxrd']
    curtime=time.time()
    rows=len(fulldf)
    q=np.log(10)/400
    logliks=0.0
    hburn=10
    startyear=fulldf['Season'].values[0]
    burn=1983-startyear
    #print(startyear)
    year=copy.deepcopy(startyear)
    startday=fulldf['Season']
    Glickodictionary=copy.deepcopy(initdictionary)
    for key in Glickodictionary:
        Glickodictionary[key][1]=maxrd
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    games=0
    firstdivs=[]
    unitedrows=[]
    #print(Glickodictionary)
    yeardicts=[]
    Poismatches=[]
    while day<=totaltime:
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        
        if len(ratingperioddf!=0):
            for key in Glickodictionary:
                Glickodictionary[key][1]=timeupdate(c, Glickodictionary[key][1], maxrd)
            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                if(row['Season']!=year):
                    for key in Glickodictionary:
                        Glickodictionary[key][1]=SeasonBoost(Glickodictionary[key][1], b**2, maxrd)
                   # yearleagues.append([row['Season'], np.mean(firsts), np.mean(seconds), np.mean(thirds), np.mean(fourths)])
                    yeardicts.append([row['Season'], h, copy.deepcopy(Glickodictionary)])                          
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                Homestats=Glickodictionary[Home]
                Awaystats=Glickodictionary[Away]
                HomeRating=Homestats[0]
                AwayRating=Awaystats[0]
                HomeRd=Homestats[1]
                AwayRd=Awaystats[1]
                HomeG=g(AwayRd**2, q)
                AwayG=g(HomeRd**2, q)
                pG=g(HomeRd**2+AwayRd**2, q)
                HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
                AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
                pE=E(HomeRating, AwayRating, pG, h, 1)
                HRes=Res(HomeGoals, AwayGoals)
                ARes=Res(AwayGoals, HomeGoals)

                Glickodictionary[Home][4]+=HomeG*(HRes-HomeE)
                Glickodictionary[Away][4]+=AwayG*(ARes-AwayE)
                Glickodictionary[Home][3]+=insideD(HomeG, HomeE)
                Glickodictionary[Away][3]+=insideD(AwayG, AwayE)
                Rs+=HRes
                Es+=pE
               # row=[Home, Away,date, year,tier, HomeRating, AwayRating, h, HomeRd, AwayRd, pG, HomeGoals, AwayGoals, HRes, pE]
               # rows2.append(row)
                if(year>startyear+burn):
                    logliks+=lik(pE, HRes)
                    games+=1
                if(year>1991):
                    if(tier==1):
                        firstdivs.append(abs(HomeRating-AwayRating+h))
                if(year>2008):
                    if(tier==1):
                        Poismatches.append([Home, Away,year,  HomeGoals, AwayGoals, HomeRating, AwayRating,HomeRd, AwayRd, h, 1])
                        Poismatches.append([Away, Home,year,  AwayGoals, HomeGoals, AwayRating, HomeRating,AwayRd, HomeRd, h, -1])

            if(year>startyear+hburn):
                h+=hk*(Rs-Es)


            
            

            for key in Glickodictionary:
                Glickod=d2(q, Glickodictionary[key][3])
                Glickodictionary[key][0]=ratingupdate(Glickodictionary[key][0],q, Glickod, Glickodictionary[key][1], Glickodictionary[key][4])
                Glickodictionary[key][1]=rdupdate(Glickodictionary[key][1], Glickod)
                Glickodictionary[key][3]=0
                Glickodictionary[key][4]=0
            unitedrows.append([date, copy.deepcopy(Glickodictionary['Manchester United'][0]),copy.deepcopy(Glickodictionary['Manchester United'][1]) ])
        #rows3.append([date, Glickodictionary['Tottenham Hotspur'][1]])  
        day+=ratingperiodlength
    print(params ,logliks,games, Glickodictionary['Tottenham Hotspur'][0], time.time()-curtime)
    #print(Glickodictionary)
    return unitedrows

In [121]:
Glickologlikboost(bestnew)

{'b': 41.78112448478919, 'c': 3.507640285224943, 'hk': 0.28147894353485076, 'maxrd': 109.05389727014936} -44179.0204871 67126 1969.04766193 25.65364408493042


[['Portsmouth',
  'Fulham',
  2009,
  0,
  1,
  1738.4546186292603,
  1753.1486397339713,
  62.832158271572268,
  62.966384388766897,
  54.198012589990654,
  1],
 ['Fulham',
  'Portsmouth',
  2009,
  1,
  0,
  1753.1486397339713,
  1738.4546186292603,
  62.966384388766897,
  62.832158271572268,
  54.198012589990654,
  -1],
 ['Bolton Wanderers',
  'Sunderland',
  2009,
  0,
  1,
  1714.575802250411,
  1672.0112074730496,
  62.932740256215773,
  63.197603187135464,
  54.198012589990654,
  1],
 ['Sunderland',
  'Bolton Wanderers',
  2009,
  1,
  0,
  1672.0112074730496,
  1714.575802250411,
  63.197603187135464,
  62.932740256215773,
  54.198012589990654,
  -1],
 ['Blackburn Rovers',
  'Manchester City',
  2009,
  0,
  2,
  1753.1547207047549,
  1754.3300584075887,
  62.830527002102471,
  62.808770590061066,
  54.198012589990654,
  1],
 ['Manchester City',
  'Blackburn Rovers',
  2009,
  2,
  0,
  1754.3300584075887,
  1753.1547207047549,
  62.808770590061066,
  62.830527002102471,
  54.1

In [101]:
def Glickologlikboost(params):
 #   yearleagues=[]
  #  rows2=[]
   # rows3=[]
    ratingperiodlength=7.0
    c=params['c']
    h=80
    hk=params['hk']
    b=params['b']
    #minrd=params['minrd']
    maxrd=params['maxrd']
    curtime=time.time()
    rows=len(fulldf)
    q=np.log(10)/400
    logliks=0.0
    hburn=10
    startyear=fulldf['Season'].values[0]
    burn=1983-startyear
    #print(startyear)
    year=copy.deepcopy(startyear)
    startday=fulldf['Season']
    Glickodictionary=copy.deepcopy(initdictionary)
    for key in Glickodictionary:
        Glickodictionary[key][1]=maxrd
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    games=0
    Poismatches=[]
   # unitedrows=[]
    #print(Glickodictionary)
    yeardicts=[]
    while day<=totaltime:

        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):
            Rs=0.
            Es=0.
            for key in Glickodictionary:
                Glickodictionary[key][1]=timeupdate(c, Glickodictionary[key][1], maxrd)
            for ind, row in ratingperioddf.iterrows():
                if(row['Season']!=year):
                    for key in Glickodictionary:
                        Glickodictionary[key][1]=SeasonBoost(Glickodictionary[key][1], b, maxrd)
                    yeardicts.append([row['Season'], h, copy.deepcopy(Glickodictionary)])       
                         
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                Homestats=Glickodictionary[Home]
                Awaystats=Glickodictionary[Away]
                HomeRating=Homestats[0]
                AwayRating=Awaystats[0]
                HomeRd=Homestats[1]
                AwayRd=Awaystats[1]
                HomeG=g(AwayRd**2, q)
                AwayG=g(HomeRd**2, q)
                pG=g(HomeRd**2+AwayRd**2, q)
                HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
                AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
                pE=E(HomeRating, AwayRating, pG, h, 1)
                HRes=Res(HomeGoals, AwayGoals)
                ARes=Res(AwayGoals, HomeGoals)

                Glickodictionary[Home][4]+=HomeG*(HRes-HomeE)
                Glickodictionary[Away][4]+=AwayG*(ARes-AwayE)
                Glickodictionary[Home][3]+=insideD(HomeG, HomeE)
                Glickodictionary[Away][3]+=insideD(AwayG, AwayE)
                Rs+=HRes
                Es+=pE
               # row=[Home, Away,date, year,tier, HomeRating, AwayRating, h, HomeRd, AwayRd, pG, HomeGoals, AwayGoals, HRes, pE]
               # rows2.append(row)
                if(year>startyear+burn):
                    logliks+=lik(pE, HRes)
                    games+=1
                if(year>2008):
                    if(tier==1):
                        Poismatches.append([Home, Away,year,  HomeGoals, AwayGoals, HomeRating, AwayRating,HomeRd, AwayRd, h, 1])
                        Poismatches.append([Away, Home,year,  AwayGoals, HomeGoals, AwayRating, HomeRating,AwayRd, HomeRd, h, -1])

            if(year>startyear+hburn):
                h+=hk*(Rs-Es)
                
            
            

            for key in Glickodictionary:
                Glickod=d2(q, Glickodictionary[key][3])
                Glickodictionary[key][0]=ratingupdate(Glickodictionary[key][0],q, Glickod, Glickodictionary[key][1], Glickodictionary[key][4])
                Glickodictionary[key][1]=rdupdate(Glickodictionary[key][1], Glickod)
                Glickodictionary[key][3]=0
                Glickodictionary[key][4]=0
            #unitedrows.append([date, copy.deepcopy(Glickodictionary['Manchester United'][0]),copy.deepcopy(Glickodictionary['Manchester United'][1]) ])
        #rows3.append([date, Glickodictionary['Tottenham Hotspur'][1]])  
        day+=ratingperiodlength
    print(params ,logliks,games, Glickodictionary['Tottenham Hotspur'][0], time.time()-curtime)
    #print(Glickodictionary)
    return Poismatches

In [108]:
bestnew

{'b': 41.78112448478919,
 'c': 3.507640285224943,
 'hk': 0.28147894353485076,
 'maxrd': 109.05389727014936}

In [122]:
BoostRows=Glickologlikboost(bestnew)
columns=['Team', 'Opponent','Year', 'Goals', 'OppGoals', 'Rating', 'OppRating','Rd', 'OppRd' ,'HF', 'HomeAway']
BoostDf=pd.DataFrame(BoostRows, columns=columns)
BoostDf['RatingDif']=BoostDf['Rating']-BoostDf['OppRating']
BoostDf['EffRd']=np.sqrt(BoostDf['Rd']**2+BoostDf['OppRd']**2)
BoostDf['RatingDifRat']=BoostDf['RatingDif']/BoostDf['EffRd']
BoostDf['EffHf']=BoostDf['HomeAway']*BoostDf['HF']

{'b': 41.78112448478919, 'c': 3.507640285224943, 'hk': 0.28147894353485076, 'maxrd': 109.05389727014936} -44179.0204871 67126 1969.04766193 26.555118083953857


In [123]:
BoostDf

,Team,Opponent,Year,Goals,OppGoals,Rating,OppRating,Rd,OppRd,HF,HomeAway,RatingDif,EffRd,RatingDifRat,EffHf
0,Portsmouth,Fulham,2009,0,1,1738.454619,1753.148640,62.832158,62.966384,54.198013,1,-14.694021,88.953053,-0.165188,54.198013
1,Fulham,Portsmouth,2009,1,0,1753.148640,1738.454619,62.966384,62.832158,54.198013,-1,14.694021,88.953053,0.165188,-54.198013
2,Bolton Wanderers,Sunderland,2009,0,1,1714.575802,1672.011207,62.932740,63.197603,54.198013,1,42.564595,89.187818,0.477247,54.198013
3,Sunderland,Bolton Wanderers,2009,1,0,1672.011207,1714.575802,63.197603,62.932740,54.198013,-1,-42.564595,89.187818,-0.477247,-54.198013
4,Blackburn Rovers,Manchester City,2009,0,2,1753.154721,1754.330058,62.830527,62.808771,54.198013,1,-1.175338,88.840401,-0.013230,54.198013
5,Manchester City,Blackburn Rovers,2009,2,0,1754.330058,1753.154721,62.808771,62.830527,54.198013,-1,1.175338,88.840401,0.013230,-54.198013
6,Stoke City,Burnley,2009,2,0,1699.071337,1631.655849,62.736781,60.438981,54.198013,1,67.415488,87.113570,0.773880,54.198013
7,Burnley,Stoke City,2009,0,2,1631.655849,1699.071337,60.438981,62.736781,54.198013,-1,-67.415488,87.113570,-0.773880,-54.198013
8,Everton,Arsenal,2009,1,6,1850.001538,1938.502551,63.026459,64.662474,54.198013,1,-88.501014,90.297121,-0.980109,54.198013
9,Arsenal,Everton,2009,6,1,1938.502551,1850.001538,64.662474,63.026459,54.198013,-1,88.501014,90.297121,0.980109,-54.198013


In [124]:
BoostDf.to_csv('BoostPoisson.csv')

In [114]:
YearGlickos=Glickologlikboost(bestnew)

{'b': 41.78112448478919, 'c': 3.507640285224943, 'hk': 0.28147894353485076, 'maxrd': 109.05389727014936} -44179.0204871 67126 1969.04766193 25.935338020324707


In [115]:
YearGlickos

[[1959,
  80,
  {'AFC Bournemouth': [1498.6227716768988, 65.787805165032381, 3, 0, 0],
   'AFC Wimbledon': [1500.0, 109.05389727014936, 4, 0, 0],
   'Accrington': [1500.0, 109.05389727014936, 4, 0, 0],
   'Accrington Stanley': [1469.7640022849282, 65.621391311685116, 3, 0, 0],
   'Aldershot': [1425.6039265067291, 65.920968046457673, 4, 0, 0],
   'Arsenal': [1553.8010473376733, 67.206099172234815, 1, 0, 0],
   'Aston Villa': [1412.0149168217263, 67.593384496052067, 1, 0, 0],
   'Barnet': [1500.0, 109.05389727014936, 4, 0, 0],
   'Barnsley': [1387.7181775611289, 67.465930962645317, 2, 0, 0],
   'Barrow': [1374.5419331008427, 66.056336490712127, 4, 0, 0],
   'Birmingham City': [1531.7867256470397, 66.919275217222463, 1, 0, 0],
   'Blackburn Rovers': [1514.2478243855576, 67.125601364111276, 1, 0, 0],
   'Blackpool': [1532.9276606960041, 66.955999188882956, 1, 0, 0],
   'Bolton Wanderers': [1557.8171843777732, 67.471781637118895, 1, 0, 0],
   'Boston United': [1500.0, 109.05389727014936, 4,

In [125]:
YearGlickos

[[1959,
  80,
  {'AFC Bournemouth': [1498.6227716768988, 65.787805165032381, 3, 0, 0],
   'AFC Wimbledon': [1500.0, 109.05389727014936, 4, 0, 0],
   'Accrington': [1500.0, 109.05389727014936, 4, 0, 0],
   'Accrington Stanley': [1469.7640022849282, 65.621391311685116, 3, 0, 0],
   'Aldershot': [1425.6039265067291, 65.920968046457673, 4, 0, 0],
   'Arsenal': [1553.8010473376733, 67.206099172234815, 1, 0, 0],
   'Aston Villa': [1412.0149168217263, 67.593384496052067, 1, 0, 0],
   'Barnet': [1500.0, 109.05389727014936, 4, 0, 0],
   'Barnsley': [1387.7181775611289, 67.465930962645317, 2, 0, 0],
   'Barrow': [1374.5419331008427, 66.056336490712127, 4, 0, 0],
   'Birmingham City': [1531.7867256470397, 66.919275217222463, 1, 0, 0],
   'Blackburn Rovers': [1514.2478243855576, 67.125601364111276, 1, 0, 0],
   'Blackpool': [1532.9276606960041, 66.955999188882956, 1, 0, 0],
   'Bolton Wanderers': [1557.8171843777732, 67.471781637118895, 1, 0, 0],
   'Boston United': [1500.0, 109.05389727014936, 4,

In [152]:
def SeasonSimulatorBoost(year):
    maxrd=109.05389727014936
    c=3.507640285224943
    ratingperiodlength=7.0
    rc=0.76987
    hc=1.11409
    intc=0.26481
    hk=0
    q=np.log(10)/400
    for i in range(len(YearGlickos)):
        if YearGlickos[i][0]==year:
            yeardict=YearGlickos[i][2]
            h=YearGlickos[i][1]
    yeardf=fulldf[fulldf['Season']==year].reset_index()
    del yeardf['index']
    premdf=yeardf[yeardf['tier']==1].reset_index()
    del premdf['index']
    del premdf['level_0']
    del premdf['Unnamed: 0']
    teams=premdf['home'].unique()
    teamdict={}
    for team in teams:
        yeardictent=yeardict[team]
        teamdict[team]=[yeardict[team][0], yeardict[team][1], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #print(teamdict)
    #Index 0: rating, index 1: rd, index 2: dcalc, index 3: scalc, index 4: wins, index5: draws, index6: losses, index7: points, index8: matches.
    #index 9: GF, index10: GA, index11: GD
    currentday=min(premdf['Day#'])
    lastday=max(premdf['Day#'])
    #print(teamdict)
    while currentday<=lastday:
        Rs=0
        Es=0
        for key in teamdict:
            teamdict[key][1]=timeupdate(c, teamdict[key][1], maxrd)
        
        ratingperioddf=premdf[(premdf['Day#'] >=currentday) & (premdf['Day#']<currentday+ratingperiodlength)].reset_index()
        for ind, row in ratingperioddf.iterrows():
            Home=row['home']
            Away=row['visitor']
            HomeRating=teamdict[Home][0]
            AwayRating=teamdict[Away][0]
            ratingdif=(HomeRating-AwayRating)/400.
            HomeLambda=np.exp(intc+ratingdif*rc+hc*h/400.)
            AwayLambda=np.exp(intc-ratingdif*rc-hc*h/400.)
            #print([Home, Away, HomeLambda, AwayLambda])
            HomeSG=np.random.poisson(HomeLambda)
            AwaySG=np.random.poisson(AwayLambda)
            teamdict[Home][8]+=1
            teamdict[Away][8]+=1
            teamdict[Home][9]+=HomeSG
            teamdict[Away][9]+=AwaySG
            teamdict[Away][10]+=HomeSG
            teamdict[Home][10]+=AwaySG
            teamdict[Home][11]+=HomeSG-AwaySG
            teamdict[Away][11]+=AwaySG-HomeSG
            if(HomeSG>AwaySG):
                teamdict[Home][4]+=1
                teamdict[Home][7]+=3
                teamdict[Away][6]+=1
            elif(HomeSG<AwaySG):
                teamdict[Away][4]+=1
                teamdict[Away][7]+=3
                teamdict[Home][6]+=1
            elif(HomeSG==AwaySG):
                teamdict[Away][5]+=1
                teamdict[Home][5]+=1
                teamdict[Home][7]+=1
                teamdict[Away][7]+=1
            HomeRd=teamdict[Home][1]
            AwayRd=teamdict[Away][1]
            HomeG=g(AwayRd**2, q)
            AwayG=g(HomeRd**2, q)
            pG=g(HomeRd**2+AwayRd**2, q)
            HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
            AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
            pE=E(HomeRating, AwayRating, pG, h, 1)
            HRes=Res(HomeSG, AwaySG)
            ARes=Res(AwaySG, HomeSG)
            teamdict[Home][3]+=HomeG*(HRes-HomeE)
            teamdict[Away][3]+=AwayG*(ARes-AwayE)
            teamdict[Home][2]+=insideD(HomeG, HomeE)
            teamdict[Away][2]+=insideD(AwayG, AwayE)
            Rs+=HRes
            Es+=pE
        
        #h+=hk*(Rs-Es)
        for key in teamdict:
            Glickod=d2(q, teamdict[key][2])
            teamdict[key][0]=ratingupdate(teamdict[key][0],q, Glickod, teamdict[key][1], teamdict[key][3])
            teamdict[key][1]=rdupdate(teamdict[key][1], Glickod)
            teamdict[key][2]=0
            teamdict[key][3]=0
        
        currentday+=ratingperiodlength
    standings=pd.DataFrame.from_dict(teamdict, orient='index')
    standings=standings.reset_index()
    standings.columns = ['Team','Rating','Rating Deviation',
                     'Filler1','Filler2','Wins', 'Losses', 'Draws', 'Points', 'GP', 'GF', 'GA', 'GD']
    del standings['Filler1']
    del standings['Filler2']
    standings=standings.sort_values(['Points', 'GD', 'GA'], ascending=False).reset_index()
    del standings['index']
    #print(teamdict)
    return list(standings['Team'])

In [138]:
SeasonSimulatorBoost(2015)

['Arsenal',
 'Manchester City',
 'Liverpool',
 'Chelsea',
 'Tottenham Hotspur',
 'Swansea City',
 'Everton',
 'Newcastle United',
 'Stoke City',
 'Manchester United',
 'Leicester City',
 'Southampton',
 'Norwich City',
 'Watford',
 'West Ham United',
 'West Bromwich Albion',
 'AFC Bournemouth',
 'Aston Villa',
 'Crystal Palace',
 'Sunderland']

In [156]:
def storedresultsb(season, n):
    sim=[]
    counts=0
    for i in range(n):
        res=SeasonSimulatorBoost(season)
        if(res[0]!='Manchester United'):
            if(res[0]!='Arsenal'):
                if(res[0]!='Liverpool'):
                    if(res[0]!='Tottenham Hotspur'):
                        if(res[0]!='Chelsea'):
                            if(res[0]!='Manchester City'):
                                if(res[0]!='Leicester City'):
                                    counts+=1
                                    if(counts%500==0):
                                        print([i, season, counts, counts/i])
        sim.append(res)
    return sim

In [157]:
results=storedresultsb(2016, 250000)

[5787, 2016, 500, 0.08640055296353896]
[11192, 2016, 1000, 0.08934953538241601]
[16317, 2016, 1500, 0.09192866335723479]
[22047, 2016, 2000, 0.09071529006213998]
[27805, 2016, 2500, 0.08991188635137565]
[33220, 2016, 3000, 0.09030704394942805]
[39266, 2016, 3500, 0.08913563897519483]
[44632, 2016, 4000, 0.08962179602079226]
[50347, 2016, 4500, 0.08937970484835243]
[56153, 2016, 5000, 0.08904243762577245]
[62058, 2016, 5500, 0.08862676850688066]
[67527, 2016, 6000, 0.08885334754986894]
[73324, 2016, 6500, 0.08864764606404452]
[78964, 2016, 7000, 0.08864799148979281]
[84898, 2016, 7500, 0.08834130368206554]
[90984, 2016, 8000, 0.08792754770069462]
[96721, 2016, 8500, 0.08788163894087117]
[102171, 2016, 9000, 0.08808761781718882]
[108033, 2016, 9500, 0.08793609360102932]
[113781, 2016, 10000, 0.08788813598052399]
[118876, 2016, 10500, 0.08832733268279552]
[124663, 2016, 11000, 0.08823788934968675]
[130178, 2016, 11500, 0.08834057982147521]
[135675, 2016, 12000, 0.08844665561083472]
[14106

In [162]:
champions=[]
topfour=[]
relegated=[]
for i in range(250000):
    champions.append(results[i][0])
    topfour.append(results[i][0])
    topfour.append(results[i][1])
    topfour.append(results[i][2])
    topfour.append(results[i][3])
    relegated.append(results[i][-1])
    relegated.append(results[i][-2])
    relegated.append(results[i][-3])
uniquechamps=sorted(set(champions))

champset=[]
relset=[]
fourset=[]
ratings=[]
rds=[]
teamdict=YearGlickos[57][2]
for i in range(len(uniquechamps)):
    ratings.append(teamdict[uniquechamps[i]][0])
    rds.append(teamdict[uniquechamps[i]][1])
    champset.append(champions.count(uniquechamps[i])/2500)
    relset.append(relegated.count(uniquechamps[i])/2500)
    fourset.append(topfour.count(uniquechamps[i])/2500)
df1 = pd.DataFrame(uniquechamps)
df2=pd.DataFrame(ratings)
df3=pd.DataFrame(rds)
df4 = pd.DataFrame(champset)
df5 = pd.DataFrame(fourset)

df6 = pd.DataFrame(relset)

df7 = pd.concat([df1, df2, df3, df4, df5, df6], join='outer', axis=1)
df7.columns = ['Team','Preseason Rating','Preseason Rating Deviation', 'Champions', 'Top Four', 'Relegation']
df7

,Team,Preseason Rating,Preseason Rating Deviation,Champions,Top Four,Relegation
0,AFC Bournemouth,1709.828771,62.273095,0.0784,1.2660,37.2112
1,Arsenal,1931.517249,63.978324,23.1472,63.5436,0.2180
2,Burnley,1750.132778,60.919147,0.2836,3.5568,20.4896
3,Chelsea,1883.173359,63.670600,9.3464,40.3484,1.1060
4,Crystal Palace,1728.091909,62.586852,0.1476,2.0808,29.3584
5,Everton,1791.651177,62.394669,1.0224,9.2568,9.7712
6,Hull City,1700.686360,60.902516,0.0556,0.9288,41.3996
7,Leicester City,1872.898517,62.270428,7.6132,35.8032,1.3568
8,Liverpool,1865.598406,62.766353,6.5808,32.1476,1.7264
9,Manchester City,1920.651016,64.409848,19.1788,58.3460,0.3468


In [40]:
teamdict

[2012,
 59.081048797048979,
 {'AFC Bournemouth': [1455.5433307465501, 60.640676582981207, 3, 0, 0],
  'AFC Wimbledon': [1347.2151870334146, 66.377336708747222, 4, 0, 0],
  'Accrington': [1362.8319148024279, 60.693468415273891, 4, 0, 0],
  'Accrington Stanley': [1338.2389635759766, 111.6204164961327, 3, 0, 0],
  'Aldershot': [1379.3188831240286, 60.630982097657515, 4, 0, 0],
  'Arsenal': [1898.7200433658659, 64.142240534193235, 1, 0, 0],
  'Aston Villa': [1753.3763384890012, 62.998289631105102, 1, 0, 0],
  'Barnet': [1281.8159813883551, 61.150246767873391, 4, 0, 0],
  'Barnsley': [1535.2378236912036, 60.790024900753899, 2, 0, 0],
  'Barrow': [1302.4660070637599, 111.6204164961327, 4, 0, 0],
  'Birmingham City': [1700.326487619541, 61.487212917192174, 1, 0, 0],
  'Blackburn Rovers': [1681.4691619479511, 63.221288636086335, 1, 0, 0],
  'Blackpool': [1670.6004395344703, 61.250678069484891, 1, 0, 0],
  'Bolton Wanderers': [1691.0476561175828, 63.357533543757739, 1, 0, 0],
  'Boston United':

In [163]:
df7.sort_values('Champions', ascending=False)


,Team,Preseason Rating,Preseason Rating Deviation,Champions,Top Four,Relegation
1,Arsenal,1931.517249,63.978324,23.1472,63.5436,0.2180
9,Manchester City,1920.651016,64.409848,19.1788,58.3460,0.3468
16,Tottenham Hotspur,1899.316232,63.001280,12.7668,48.1224,0.6328
10,Manchester United,1897.797357,63.403903,12.5248,47.3000,0.6912
3,Chelsea,1883.173359,63.670600,9.3464,40.3484,1.1060
7,Leicester City,1872.898517,62.270428,7.6132,35.8032,1.3568
8,Liverpool,1865.598406,62.766353,6.5808,32.1476,1.7264
12,Southampton,1837.804556,62.382335,3.3992,21.3728,3.4716
19,West Ham United,1812.847117,62.400210,1.8132,13.8152,6.2552
5,Everton,1791.651177,62.394669,1.0224,9.2568,9.7712


In [165]:
from scipy.stats import spearmanr

premdict={}
premdict['Chelsea']=0
premdict['Tottenham Hotspur']=1
premdict['Manchester City']=2
premdict['Liverpool']=3
premdict['Arsenal']=4
premdict['Manchester United']=5 
premdict['Everton']=6
premdict['Southampton']=7
premdict['AFC Bournemouth']=8
premdict['West Bromwich Albion']=9
premdict['West Ham United']=10
premdict['Leicester City']=11
premdict['Stoke City']=12
premdict['Crystal Palace']=13
premdict['Swansea City']=14
premdict['Burnley']=15 
premdict['Watford']=16
premdict['Hull City']=17
premdict['Middlesbrough']=18
premdict['Sunderland']=19
spearmans=[]
spearmancorrs=[]
for j in range(len(results)):
    if(j%2500==0):
        print(j)
    templist=[]
    for i in range(len(results[j])):
        #print(templist)
        templist.append(premdict[results[j][i]])
    corr, p_value=spearmanr(templist,list(range(0, 20)))
    spearmans.append([corr, p_value])
    spearmancorrs.append(corr)

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
200000
202500
205000
207500
210000
212500
215000
217500
220000
222500
225000
227500
230000
232500
235000
237500
240000
242500
245000
247500


In [166]:
np.mean(spearmancorrs)


0.5614053954887217

In [49]:
np.std(spearmancorrs)


0.12218801868659747

In [167]:
lik=0
for key in premdict:
    count=0
    for i in range(len(results)):
        if(results[i][premdict[key]]==key):
            count+=1

    lik+=np.log(count/250000)
lik

-50.947938817982731

In [54]:
g(55**2, np.log(10)/400)

0.98510502216541707

In [168]:
results

[['Manchester City',
  'Everton',
  'West Ham United',
  'Liverpool',
  'Arsenal',
  'Manchester United',
  'Swansea City',
  'Middlesbrough',
  'Southampton',
  'Tottenham Hotspur',
  'Leicester City',
  'Watford',
  'Hull City',
  'AFC Bournemouth',
  'Chelsea',
  'Crystal Palace',
  'Sunderland',
  'Stoke City',
  'Burnley',
  'West Bromwich Albion'],
 ['Tottenham Hotspur',
  'Manchester City',
  'Liverpool',
  'Manchester United',
  'Chelsea',
  'Crystal Palace',
  'Arsenal',
  'Hull City',
  'Southampton',
  'West Ham United',
  'Swansea City',
  'West Bromwich Albion',
  'Leicester City',
  'Stoke City',
  'AFC Bournemouth',
  'Watford',
  'Everton',
  'Sunderland',
  'Burnley',
  'Middlesbrough'],
 ['Manchester City',
  'Arsenal',
  'Tottenham Hotspur',
  'Liverpool',
  'Leicester City',
  'Manchester United',
  'Southampton',
  'Everton',
  'West Ham United',
  'Watford',
  'Sunderland',
  'Chelsea',
  'West Bromwich Albion',
  'Burnley',
  'Stoke City',
  'Swansea City',
  'Mi

In [143]:
masterresults=[]
nSims=100000
for i in range(2009, 2017, 1):
    masterresults.append([i, storedresults(i, nSims)])

NameError: name 'storedresults' is not defined

In [151]:
masterresults=[]
nSims=100000
for i in range(2009, 2017, 1):
    masterresults.append([i, storedresultsb(i, nSims)])

[7002, 2009, 100, 0.014281633818908884]
[12947, 2009, 200, 0.015447594037228701]
[19360, 2009, 300, 0.015495867768595042]
[25942, 2009, 400, 0.01541901164135379]
[32180, 2009, 500, 0.015537600994406464]
[37758, 2009, 600, 0.01589067217543302]
[45092, 2009, 700, 0.015523817972145835]
[51751, 2009, 800, 0.015458638480415838]
[59796, 2009, 900, 0.015051173991571343]
[66541, 2009, 1000, 0.015028328399032176]
[72681, 2009, 1100, 0.015134629407960815]
[79056, 2009, 1200, 0.015179113539769277]
[86667, 2009, 1300, 0.0149999423079142]


KeyboardInterrupt: 

In [202]:
bestnew

{'b': 41.78112448478919,
 'c': 3.507640285224943,
 'hk': 0.28147894353485076,
 'maxrd': 109.05389727014936}